In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import threading
from bs4 import BeautifulSoup
from fake_useragent import UserAgent 
import requests
import pandas as pd

In [38]:
# 상세 페이지 

MUSINSA_COAT_LIST_URL ="https://www.musinsa.com/categories/item/002008"
response =requests.get(MUSINSA_COAT_LIST_URL)


def get_chrome_driver_useragent():
    agent = UserAgent().chrome
    chrome_options = webdriver.ChromeOptions()
    
    chrome_options.add_argument('user-agent={}'.format(UserAgent().chrome)) # 크롬을 이용하지만 user을 속일수있음
    
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )
    
    return driver

picture_url_list=[]

coat_info=[]
for i in range(11,13):
    
    driver = get_chrome_driver_useragent()
    driver.get(MUSINSA_COAT_LIST_URL)

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    coat_title =soup.select_one(f'#searchList > li:nth-child({i}) > div.li_inner > div.article_info > p.item_title').text
    coat_name = soup.select_one(f'#searchList > li:nth-child({i}) > div.li_inner > div.article_info > p.list_info > a').text.replace('\n','').strip()
    coat_price1 = soup.select_one(f'#searchList > li:nth-child({i}) > div.li_inner > div.article_info > p.price > del')
    coat_price2 = soup.select_one(f'#searchList > li:nth-child({i}) > div.li_inner > div.article_info > p.price' )
    try:
        coat_price = coat_price2.text.replace(coat_price1.text,'').replace('\n','').strip()
    except:
        coat_price = soup.select_one(f'#searchList > li:nth-child({i}) > div.li_inner > div.article_info > p.price' ).text.replace('\n','').strip()
    try:
        coat_star = soup.select_one(f'#searchList > li:nth-child({i}) > div.li_inner > div.article_info > p.point > span.count').text
    except:
        coat_star=None
    try:
        coat_heart = soup.select_one(f'#searchList > li:nth-child({i}) > div.li_inner > div.article_info > p.txt_cnt_like').text.strip()
    except:
        coat_heart=None
    
    coat_detail_URL = soup.select_one(f'#searchList > li:nth-child({i}) > div.li_inner > div.list_img > a')['href']
    
    coat_detail_URL='https:'+ coat_detail_URL
    driver.get(coat_detail_URL)
    #response =requests.get(coat_detail_URL)


    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    coat_season =soup.select_one('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(2) > p.product_article_contents > strong').text.replace(' ','').strip()
    coat_gender=soup.select_one('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(2) > p.product_article_contents > span.txt_gender > span').text
    try:
        coat_like =soup.select_one('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(6) > p.product_article_contents > a > span.prd-score__rating').text
    except:
        coat_like=None
    j=0
    while True:
        picture=[]
        try:
            coat_picture_url="https:"+soup.select('#detail_thumb > ul > li > img')[j]['src']
            picture.append(coat_picture_url)
            j+=1
        except:
            break
        picture_url_list.append(picture)
        
    coat_info.append([coat_title,coat_name,coat_season,coat_price,coat_star,coat_heart,coat_gender,coat_like,picture_url_list])
    driver.quit()
    

print(coat_info)


[['블랭크03', 'classic half trench coat (olive ...', '2023S/S', '298,000원', None, '6', '여', None, [['https://image.msscdn.net/images/goods_img/20230324/3176866/3176866_16796229794032_60.jpg'], ['https://image.msscdn.net/images/goods_img/20230321/3165161/3165161_16793723394694_60.jpg'], ['https://image.msscdn.net/images/prd_img/20230321/3165161/detail_3165161_16793723474866_60.jpg']]], ['올젠', '프리즘 맥코트_ZAD1CG1901', '2023S/S', '298,000원', None, '2', '남', None, [['https://image.msscdn.net/images/goods_img/20230324/3176866/3176866_16796229794032_60.jpg'], ['https://image.msscdn.net/images/goods_img/20230321/3165161/3165161_16793723394694_60.jpg'], ['https://image.msscdn.net/images/prd_img/20230321/3165161/detail_3165161_16793723474866_60.jpg']]]]


In [37]:
df=pd.DataFrame(coat_info,index=['11','12'])
df



,0,1,2,3
11,2023S/S,여,None,[[https://image.msscdn.net/images/goods_img/20...
12,2023S/S,남,None,[[https://image.msscdn.net/images/goods_img/20...
